In [12]:
import numpy as np
import random
from sympy import symbols, diff, solve, sqrt, cos, sin, pi

### Finding Roots

#### Newton Raphson

In [ ]:
def newtonraphson(f, df, x0, maxitr=500, tol=1e-20):
# Input f as well as f'
  i = 0
  error = 1
  while i < maxitr and error >= tol:
    x1 = x0 - f(x0) / df(x0)
    error = abs(x1 - x0)
    i += 1
    x0 = x1
  return x0

#### Secant Method

In [7]:
def secant(f, x1, x2, maxitr = 500, tol = 1e-20):
  error = 1
  i = 0
  while (i < maxitr and error >= tol):
    x3 = (x1*f(x2) - x2*f(x1))/(f(x2)-f(x1))
    error = abs(x3-x2)
    i += 1
    x1,x2 = x2,x3
  return x2

#### Regula Falsi

In [ ]:
def regulafalsi(f, x1, x2, maxitr=500, tol=1e-20):
    i = 0
    error = 1
    while i < maxitr and error >= tol:
        x3 = (x1 * f(x2) - x2 * f(x1)) / (f(x2) - f(x1))
        if f(x1) * f(x3) > 0:
            error = abs(x3 - x2)
            x1, x2 = x2, x3
        elif f(x2) * f(x3) > 0:
            error = abs(x3 - x1)
            x1, x2 = x1, x3
        i += 1
    return x2

### Curve Fitting - Least Square Methods

In [ ]:
def leastsquares(x, y, degree):
    N = len(x)
    X = np.vander(x, degree + 1, increasing=True)
    b = np.linalg.inv(X.T @ X) @ (X.T @ y)
    return b

### System of Linear Equations - Iterative

#### Jacobi Method

In [8]:
def jacobi(A, b, x0, tol=1e-10, maxitr=3000):
    n = A.shape[0]
    x1 = np.zeros_like(x0)
    for k in range(maxitr):
        for i in range(n):
            sum = 0
            for j in range(n):
                if j != i:
                    sum += A[i, j] * x0[j]
            x1[i] = (b[i] - sum) / A[i, i]
        if np.linalg.norm(x1 - x0, ord=np.inf) < tol:
            return x1
        x0 = x1.copy()
    return x1


#### Gauss Seidel Method

In [9]:
def gaussseidel(A, b, x0, tol=1e-20,maxitr=1000):
    n = len(b)
    for k in range(maxitr):
        x1 = np.copy(x0)
        for i in range(n):
            s1 = 0
            s2 = 0
            for j in range(n):
                if j < i:
                    s1 += A[i, j] * x1[j]
                elif j > i:
                    s2 += A[i, j] * x0[j]
                x1[i] = (b[i] - s1 - s2) / A[i][i]
            k += 1
        if np.linalg.norm(A@x1-b) < tol:
            return x1
    return x0

## One Dimensional Search Method

#### Bracketing Minimizer

In [20]:
def bracketmini(f,x,a,b,h):
    rho = (1+np.sqrt(5))/2
    if f(x) > f(a) and f(b) > f(a):
        return x, b

    while f(x) < f(a) and f(b) < f(a):
        if f(x)< f(a) and f(a) < f(b):
            xx = x - h
            h = h*rho
            x = xx
        elif f(x) > f(a) and f(a) > f(b):
            xx = b + h
            h = h*rho
            b = xx
    return x, b

In [6]:
def eism(f, a, b, tol=1e-5):
    if (b-a) <= tol:
        if f(a) < f(b):
            return a
        else:
            return b
    x1 = a + (b-a)/3
    x2 = b - (b-a)/3
    f1 = f(x1)
    f2 = f(x2)
    while (b-a) > tol:
        if f1 < f2:
            b = x2
            x1 = a + (b-a)/3
            x2 = b - (b-a)/3
            f1 = f(x1)
            f2 = f(x2)
        else:
            a = x1
            x1 = a + (b-a)/3
            x2 = b - (b-a)/3
            f1 = f(x1)
            f2 = f(x2)
    if f1 < f2:
        return x1
    else:
        return x2

#### Golden Ration MEthod

In [22]:
phi = (1+np.sqrt(5))/2
def grm(f, a, b, tol = 1e-10):
    rho = 2-phi
    x = (a+b)/2
    a,b = bracketmini(f,x,a,b,0.1)
    while b - a > tol:
        x1 = a + (b-a)*(rho)
        x2 = b - (b-a)*(rho)
        if f(x1) < f(x2):
            b = x2
        else:
            a = x1
    return (a+b)/2

#### Fibonacci Search method

In [21]:
def fsm(f, a, b, rho = 2-phi, tol = 1e-20):
    x = (a+b)/2
    a,b = bracketmini(f,x,a,b,0.1)
    x1 = a + (b-a)*(rho)
    x2 = b - (b-a)*(rho)
    while b - a > tol:
        rho = 1 - (rho/(1-rho))
        print((a+b)/2)
        if rho == 0.5:
                return (a+b)/2
        if f(x1) < f(x2):
            b, x2 = x2, x1
            x1 = a + (b - a) * rho
        else:
            a, x1 = x1, x2
            x2 = b - (b - a) * rho
    return (a+b)/2

#### Quadratic Interpolation MEthod

In [ ]:
def qim(f, a, b, tol = 1e-20):
    c = (a+b)/2
    lam = 0.5*(f(a)*(b**2-c**2) + f(b)*(c**2-a**2) + f(c)*(a**2-b**2))/(f(a)*(b-c) + f(b)*(c-a) + f(c)*(a-b))
    while b - a > tol:
        lam = 0.5*(f(a)*(b**2-c**2) + f(b)*(c**2-a**2) + f(c)*(a**2-b**2))/(f(a)*(b-c) + f(b)*(c-a) + f(c)*(a-b))
        print(lam)
        if f(lam) < f(c):
            c, b = lam, c
        else:
            a, c = lam, c
    return lam

### Gradient Based Method

In [ ]:
def rwalk(f, x0, y0, epsilon=1e-5, max_iter=1000):
    x, y, t = symbols('x y t')
    gradx = diff(f, x)
    grady = diff(f, y)
    def gradient(xval, yval):
        return [gradx.evalf(subs={x: xval, y: yval}), grady.evalf(subs={x: xval, y: yval})]
    def step_length(x_val, y_val, direction):
        t = symbols('t')
        g = f.subs({x: x_val + t * direction[0], y: y_val + t * direction[1]})
        T = solve(diff(g, t) == 0, t)
        return float(T[0]) if T else 1.0
    xn, yn = x0, y0
    for _ in range(max_iter):
        grad = gradient(xn, yn)
        norm_grad = sqrt(grad[0]**2 + grad[1]**2)
        if norm_grad < epsilon:
            break
        rangle = random.uniform(0, 2 * pi)
        rdir = [cos(rangle), sin(rangle)]
        stepsize = step_length(xn, yn, rdir)
        direction = [-grad[0] + stepsize * rdir[0], -grad[1] + stepsize * rdir[1]]
        norm = sqrt(direction[0]**2 + direction[1]**2)
        direction = [di / norm for di in direction]
        xno = xn + stepsize * direction[0]
        yno = yn + stepsize * direction[1]
        xn, yn = xno, yno
    return xn, yn

#### Univariate Search method

In [24]:
import numpy as np

def usme1(f, x0, y0, e = 0.01):
    z0 = (x0, y0)
    f0 = f(*z0)
    e1 = (1, 0)
    fxp = f(z0[0] + e * e1[0], z0[1])
    fxn = f(z0[0] - e * e1[0], z0[1])
    if fxp < fxn:
        if fxp < f0:
            def g(t):
                return f(z0[0] + t * e1[0], z0[1] + t * e1[1])
            a, b = bracketmini(g, 0, -100, 100, 0.1)
            T = fsm(g, a, b, rho = 2 - np.sqrt(5) / 2, tol = 1e-20)
            if T is not None:
                z1 = (z0[0] + T * e1[0], z0[1] + T * e1[1])
                return z1
    elif fxn < fxp:
        if fxn < f0:
            def g(t):
                return f(z0[0] - t * e1[0], z0[1] - t * e1[1])
            a, b = bracketmini(g, 0, -100, 100, 0.1)
            T = fsm(g, a, b, rho = 2 - np.sqrt(5) / 2, tol = 1e-20)
            if T is not None:
                z1 = (z0[0] - T * e1[0], z0[1] - T * e1[1])
                return z1
    return z0

def usme2(f, x0, y0, e = 0.01):
    z0 = (x0, y0)
    f0 = f(*z0)
    e2 = (0, 1)
    fyp = f(z0[0], z0[1] + e * e2[1])
    fyn = f(z0[0], z0[1] - e * e2[1])
    if fyp < fyn:
        if fyp < f0:
            def g(t):
                return f(z0[0] + t * e2[0], z0[1] + t * e2[1])
            a, b = bracketmini(g, 0, -100, 100, 0.1)
            T = fsm(g, a, b, rho = 2 - np.sqrt(5) / 2, tol = 1e-20)
            if T is not None:
                z1 = (z0[0] + T * e2[0], z0[1] + T * e2[1])
                return z1
    elif fyn < fyp:
        if fyn < f0:
            def g(t):
                return f(z0[0] - t * e2[0], z0[1] - t * e2[1])
            a, b = bracketmini(g, 0, -100, 100, 0.1)
            T = fsm(g, a, b, rho = 2 - np.sqrt(5) / 2, tol = 1e-20)
            if T is not None:
                z1 = (z0[0] - T * e2[0], z0[1] - T * e2[1])
                return z1
    return z0

def univariatesearch(f, z0, usme1, usme2, maxitr=100, tol=1e-10):
    i = 0
    error = float('inf')
    while i < maxitr and error > tol:
        z1 = usme1(f, z0[0], z0[1])
        z2 = usme2(f, z1[0], z1[1])
        if z1 is None or z2 is None:
            print("Error: one of the intermediate points is None.")
            break
        error = np.sqrt((z1[0] - z2[0])**2 + (z1[1] - z2[1])**2)
        z0 = z2
        i += 1
        print(z0)
    return z0

KeyboardInterrupt: 

#### Powell's Method

In [25]:
def special(f, z0, usme1, usme2):
        z1 = usme1(f, z0[0], z0[1])
        z2 = usme2(f, z1[0], z1[1])
        error = ((z2[0] - z0[0])^2 + (z2[1] - z0[1])^2)^0.5
        return (z2[0] - z0[0], z2[1] - z0[1]), error

def powell(f, z0, usme1, usme2, special, maxitr = 100, tol = 1e-20):
    i = 0
    error = float('inf')
    z_final = 0
    while i < maxitr and error > tol:
        sp, error = special(f, z0, usme1, usme2)
        if sp is None:
            return z0
            break
        def g(t):
            return f(z0[0] + t * sp[0], z0[1] + t * sp[1])
        a,b = bracketmini(g, -10, 0 , 10, 0.1)
        T = fsm(g, a, b, 1e-5, 0.33)
        if T:
            l = T
            print(f"l{l}")
            z3 = (z0[0] + l * sp[0], z0[1] + l * sp[1])
            z0 = z3
            z_final = z3
        else:
            break
        i += 1
    return z0

#### Nedler Mead MEthod of Simplex

In [26]:
def assignsimplexvalues(simplex, f):
    f_values = np.array([f(*vertex) for vertex in simplex])
    indices = np.argsort(f_values)
    sorted_simplex = np.array(simplex)[indices]
    sorted_f_values = f_values[indices]
    B = sorted_simplex[0]
    G = sorted_simplex[1]
    W = sorted_simplex[2]
    fB = sorted_f_values[0]
    fG = sorted_f_values[1]
    fW = sorted_f_values[2]
    return B, G, W, fB, fG, fW

def neldermead(f, initial_simplex, tol=1e-10, max_iter=500):
    simplex = np.array(initial_simplex)
    f_values = np.array([f(*vertex) for vertex in simplex])
    error = tol + 1
    for i in range(max_iter):
        if error <= tol:
            break
        B, G, W, fB, fG, fW = assignsimplexvalues(simplex, f)
        M = (B + G) / 2
        R = 2 * M - W  # reflect pt
        fR = f(*R)
        if fR < fB:
            E = 2 * R - M  # ext
            fE = f(*E)
            if fE < fB:
                W = E
            else:
                W = R
        else:
            if fR < fG:
                W = R
            elif fR > fG:
                c1 = (M + R) / 2
                c2 = (M + W) / 2
                C = c1 if f(*c1) < f(*c2) else c2
                if f(*C) > fW:  # contract
                    S = (B + W) / 2
                    G = (B + G) / 2
                    W = S  # shrink
                else:
                    W = C
        simplex = [B, G, W]
        error = np.linalg.norm(W - B)
    return B, f(*B)
